In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import data_preprocessing as util
import lstm_model as lstm
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error
tf.compat.v1.set_random_seed(1)

In [ ]:
# ------------------------------------------- Hyperparameters -------------------------------------------
n_steps = 50
n_features = 2
n_lstm_units_1 = 50
alpha_1 = 0.1
n_lstm_units_2 = 20
alpha_2 = 0.1
n_dense_units = 10
act_func = 'tanh'

n_epochs = 10

d_sample = 1
gauss_sigma = 10

In [ ]:
# ------------------------------------------- Prepare Training Data -------------------------------------------
# single profile
train_cur_inv = np.loadtxt('../data/fobss_data/data/osc_19_11_18/inverter/Inverter_Current.csv', delimiter=';')
train_cur_inv = train_cur_inv[:,1]
train_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/osc_19_11_18/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
train_volt_slave_0_cell_4 = train_volt_slave_0_cell_4[:,4]
train_volt_repeat = np.full(shape=train_volt_slave_0_cell_4.shape[0], fill_value=train_volt_slave_0_cell_4[0], dtype=np.float)

X1_train, y_train_single, _ = util.prepare_data(train_cur_inv, train_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)

inv_cum = np.cumsum(train_cur_inv)
X2_train, _, _ = util.prepare_data(inv_cum, train_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)

X_train_single = np.append(X1_train, X2_train, axis=2)

print('Single Profile:', X_train_single.shape, y_train_single.shape)

# multiple profiles 
cur_profile_1 = np.loadtxt('../data/fobss_data/data/Profile 10A/inverter/Inverter_Current.csv', delimiter=';')
cur_profile_1 = cur_profile_1[:,1]
volt_profil_1 = np.loadtxt('../data/fobss_data/data/Profile 10A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
volt_profil_1 = volt_profil_1[:,4]

cur_profile_2 = np.loadtxt('../data/fobss_data/data/Profile -10A/inverter/Inverter_Current.csv', delimiter=';')
cur_profile_2 = cur_profile_2[:,1]
volt_profil_2 = np.loadtxt('../data/fobss_data/data/Profile -10A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
volt_profil_2 = volt_profil_2[:,4]

cur_profile_3 = np.loadtxt('../data/fobss_data/data/Profile 25A/inverter/Inverter_Current.csv', delimiter=';')
cur_profile_3 = cur_profile_3[:,1]
volt_profil_3 = np.loadtxt('../data/fobss_data/data/Profile 25A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
volt_profil_3 = volt_profil_3[:,4]

cur_profile_4 = np.loadtxt('../data/fobss_data/data/Profile -25A/inverter/Inverter_Current.csv', delimiter=';')
cur_profile_4 = cur_profile_4[:,1]
volt_profil_4 = np.loadtxt('../data/fobss_data/data/Profile -25A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
volt_profil_4 = volt_profil_4[:,4]

# align and cut multiple profiles
cur_profile_1 = util.align(cur_profile_1, cur_profile_1)
cur_profile_1 = cur_profile_1[:2000]
volt_profile_1 = volt_profil_1[:2000]

cur_profile_2 = util.align(cur_profile_2, cur_profile_2)
cur_profile_2 = cur_profile_2[:2000]
volt_profile_2 = volt_profil_2[:2000]

cur_profile_3 = util.align(cur_profile_3, cur_profile_3)
cur_profile_3 = cur_profile_3[:2000]
volt_profile_3 = volt_profil_3[:2000]

cur_profile_4 = util.align(cur_profile_4, cur_profile_4)
cur_profile_4 = cur_profile_4[:2000]
volt_profile_4 = volt_profil_4[:2000]

# shape into correct input format
stacked_prof_volt = np.stack([volt_profile_1, volt_profile_2, volt_profile_3, volt_profile_4], axis=0).flatten()
stacked_prof_cur = np.stack([cur_profile_1, cur_profile_2, cur_profile_3, cur_profile_4], axis=0).flatten()

X1_train, y_train, _ = util.prepare_data(stacked_prof_cur, stacked_prof_volt, False, d_sample, n_steps, gauss_sigma)

# prepare second input data
stacked_prof_volt = np.stack([volt_profile_1, volt_profile_2, volt_profile_3, volt_profile_4], axis=0).flatten()
stacked_prof_cur = np.stack([cur_profile_1, cur_profile_2, cur_profile_3, cur_profile_4], axis=0).flatten()
cum_prof_cur = np.cumsum(stacked_prof_cur)
X2_train, _, _ = util.prepare_data(cum_prof_cur, stacked_prof_volt, False, d_sample, n_steps, gauss_sigma)

X_train = np.append(X1_train, X2_train, axis=2)
print('Multiple Profiles:', X_train.shape, y_train.shape)

In [ ]:
model = lstm.initialize(n_steps, n_features, n_lstm_units_1, alpha_1, alpha_2, n_lstm_units_2, act_func)


In [ ]:
# ------------------------------------------- Train LSTM -------------------------------------------
model.compile(optimizer = 'adam', loss = 'mse', metrics=["mae"])

# Fitting the LSTM to the Training set
history = model.fit(X_train_single, y_train_single, epochs = n_epochs, verbose = 1)

In [ ]:
# ------------------------------------------- Visualize Training -------------------------------------------
loss = history.history['loss']
metrics = history.history['mae']
epochs = range(1,len(loss)+1)

# plot graph
plt.subplots(figsize = (5,5))
plt.subplot(2,1,1)
plt.plot(epochs,loss,'-o',label='training loss')
plt.legend()
plt.subplot(2,1,2)
plt.plot(epochs,metrics,'-o', color='green',label='absolute error')
plt.legend()

In [ ]:
# ------------------------------------------- Prepare Test Data -------------------------------------------
# load data
test_cur_inv = np.loadtxt('../data/fobss_data/data/osc_19_11_18/inverter/Inverter_Current.csv', delimiter=';')
test_cur_inv = test_cur_inv[:,1]
test_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/osc_19_11_18/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
test_volt_slave_0_cell_4 = test_volt_slave_0_cell_4[:,4]

# prepare prediction
X1_test1, y_test1, scaler_y1 = prepare_data(test_cur_inv, test_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)
inv_cum1 = np.cumsum(test_cur_inv)
X2_test1, _, _ = prepare_data(inv_cum1, test_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)

X_test1 = np.append(X1_test1, X2_test1, axis=2)

print(X_test1.shape, y_test1.shape)

# load data
test_cur_inv = np.loadtxt('../data/fobss_data/data/stairs_19_11_18/inverter/Inverter_Current.csv', delimiter=';')
test_cur_inv = test_cur_inv[:,1]
test_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/stairs_19_11_18/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
test_volt_slave_0_cell_4 = test_volt_slave_0_cell_4[:,4]

# prepare prediction
X1_test2, y_test2, scaler_y2 = prepare_data(test_cur_inv, test_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)
inv_cum2 = np.cumsum(test_cur_inv)
X2_test2, _, _ = prepare_data(inv_cum2, test_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)

X_test2 = np.append(X1_test2, X2_test2, axis=2)

print(X_test2.shape, y_test2.shape)

# load data
test_cur_inv = np.loadtxt('../data/fobss_data/data/Profile 10A 3x/inverter/Inverter_Current.csv', delimiter=';')
test_cur_inv = test_cur_inv[:,1]
test_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/Profile 10A 3x/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
test_volt_slave_0_cell_4 = test_volt_slave_0_cell_4[:,4]

# prepare prediction
X1_test3, y_test3, scaler_y3 = prepare_data(test_cur_inv, test_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)
inv_cum3 = np.cumsum(test_cur_inv)
X2_test3, _, _ = prepare_data(inv_cum3, test_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)

X_test3 = np.append(X1_test3, X2_test3, axis=2)

print(X_test3.shape, y_test3.shape)

In [ ]:
# ------------------------------------------- Predict on Test Data -------------------------------------------
yhat1 = model.predict(X_test1, verbose = 1)
yhat_rescaled1 = scaler_y1.inverse_transform(yhat1)
y_test_unscaled1 = scaler_y1.inverse_transform(y_test1)

yhat2 = model.predict(X_test2, verbose = 1)
yhat_rescaled2 = scaler_y2.inverse_transform(yhat2)
y_test_unscaled2 = scaler_y2.inverse_transform(y_test2)

yhat3 = model.predict(X_test3, verbose = 1)
yhat_rescaled3 = scaler_y3.inverse_transform(yhat3)
y_test_unscaled3 = scaler_y3.inverse_transform(y_test3)

# compute train, test and validation error
train_error = mean_squared_error(y_train_single, model.predict(X_train_single, verbose = 1))
validation_error = mean_squared_error(y_test_unscaled2, yhat_rescaled2)
test_error = mean_squared_error(y_test_unscaled3, yhat_rescaled3)

print('Train error:', round(loss[-1], 6))
print('Validation error:', round(validation_error, 6))
print('Test error', round(test_error, 6))

# plot profiles results
plt.subplots(figsize = (7,10))
plt.subplot(3,1,1)
plt.plot(yhat_rescaled3, color='red', label = 'predicted')
plt.plot(y_test_unscaled3, color='blue', label = 'measured')
plt.legend()
plt.subplot(3,1,2)
plt.plot(yhat_rescaled1, color='red', label = 'predicted')
plt.plot(y_test_unscaled1, color='blue', label = 'measured')
plt.legend()
plt.subplot(3,1,3)
plt.plot(yhat_rescaled2, color='red', label = 'predicted')
plt.plot(y_test_unscaled2, color='blue', label = 'measured')
plt.legend()
plt.show()